NAS implementation


Genetic Algorithm for find the best neural architecture

Create population

In [ ]:
import random
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt

In [ ]:
# Define variable type_feature_extraction for feature extraction
type_feature_extraction = "feature extraction"

# Define variable hidden_layer for hidden layer
hidden_layer = "hidden layer"

# Define variable type_activation_function for activation function
type_activation_function = "activation function"

# Define variable fitness to store the fitness value
fitness = 'fitness'

# Define variable num_of_generations to set the number of generations
num_of_generations = 10

# Define variable num_of_chromosomes to set the number of chromosomes in each generation
num_of_chromosomes = 10

# Define variable num_epochs to set the number of epochs
num_epochs = 5

# Define variable num_exe to set the number of times the training will be executed
num_exe = 5

# Define variable crossover_rate to select parents for crossover with this rate
crossover_rate = 0.8 

# Define variable mutation_rate for the rate of mutation
mutation_rate = 0.01

# Define variable chart_fitness as an empty list to store the fitness values for each generation
chart_fitness = []


In [ ]:
def chart():
    # Generate x-axis values using range() function
    x = range(len(chart_fitness))

    # Create line plot
    plt.plot(x, chart_fitness)

    # Add labels and title
    plt.xlabel('Generation')
    plt.ylabel('AVG Fitness')
    plt.title('Average of fitness of chromosomes')

    # Show the plot
    plt.show()


In [ ]:
def create_chromosomes(num_chromosomes):
    chromosomes = []
    for i in range(num_chromosomes):
        chro = dict()
        chro[type_feature_extraction] = random.randint(1,3)
        num_hidden_layer = random.randint(0, 2)
        chro[hidden_layer] = [random.choice([10, 20, 30]) for _ in range(num_hidden_layer)]
        chro[type_activation_function] = [random.choice([1, 2]) for _ in range(num_hidden_layer + 1)]
        chro[fitness] = 0.0
        chromosomes.append(chro)

    return chromosomes


In [ ]:
print(create_chromosomes(4))

[{'feature extraction': 1, 'hidden layer': [], 'activation function': [1], 'fitness': 0.0}, {'feature extraction': 2, 'hidden layer': [20], 'activation function': [1, 1], 'fitness': 0.0}, {'feature extraction': 1, 'hidden layer': [30], 'activation function': [2, 1], 'fitness': 0.0}, {'feature extraction': 3, 'hidden layer': [20], 'activation function': [1, 2], 'fitness': 0.0}]


## Models

In [ ]:
class MLP_0hidden(nn.Module):
    def __init__(self, output_size, activation):
        super(MLP_0hidden, self).__init__()
        self.fc1 = nn.LazyLinear(output_size)
        # self.activation_output = nn.ReLU() if torch.rand(1) > 0.5 else nn.Sigmoid()
        if activation[0] == 1:
            self.activation_input = nn.ReLU()
        elif activation[0] == 2:
            self.activation_input = nn.Sigmoid()
        self.activation_output = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.activation_input(x)
        x = self.fc1(x)
        x = self.activation_output(x)
        return x
    

class MLP_1hidden(nn.Module):
    def __init__(self, hidden_layer, activations):
        hidden_size = hidden_layer[0]
        output_size = 10
        super(MLP_1hidden, self).__init__()
        self.fc1 = nn.LazyLinear(hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
        if activations[0] == 1:
            self.activation_input = nn.ReLU()
        elif activations[0] == 2:
            self.activation_input = nn.Sigmoid()
        
        if activations[1] == 1:
            self.activation_hidden1 = nn.ReLU()
        elif activations[1] == 2:
            self.activation_hidden1 = nn.Sigmoid()
        
        self.activation_output  = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.activation_input(x)
        x = self.fc1(x)
        x = self.activation_hidden1(x)
        x = self.fc2(x)
        x = self.activation_output(x)
        return x
    


class MLP_2hidden(nn.Module):
    def __init__(self, hidden_layer, activations):
        super(MLP_2hidden, self).__init__()
        hidden1_size = hidden_layer[0]
        hidden2_size = hidden_layer[1]
        output_size = 10
        self.fc1 = nn.LazyLinear(hidden1_size)
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.fc3 = nn.Linear(hidden2_size, output_size)
        
        if activations[0] == 1:
            self.activation_input = nn.ReLU()
        elif activations[0] == 2:
            self.activation_input = nn.Sigmoid()
            
        if activations[1] == 1 :
            self.activation_hidden1 = nn.ReLU()
        elif activations[1] == 2:
            self.activation_hidden1 = nn.Sigmoid()
        
        if activations[2] == 1:
            self.activation_hidden2 = nn.ReLU()
        elif activations[2] == 2:
            self.activation_hidden2 = nn.Sigmoid()
        
        
        
        self.activation_output = nn.Softmax(dim=1)        

    def forward(self, x):
        x = self.activation_input(x)
        x = self.fc1(x)
        x = self.activation_hidden1(x)
        x = self.fc2(x)
        x = self.activation_hidden2(x)
        x = self.fc3(x)
        x = self.activation_output(x)
        return x




In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)

criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(model, feature_extraction, train_loader):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    model = model.to(device)
    feature_extraction = feature_extraction.to(device)
    feature_extraction.eval()  # Set to evaluation mode (no gradient calculations)

    features_labels = []
    with torch.no_grad():
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            features = feature_extraction(images)
            features = features.view(features.size(0), -1)
            features_labels.append([features, labels])

    train_accuracy = []

    for exe in range(num_exe):
        for epoch in range(num_epochs):
            train_loss = 0.0
            correct = 0
            total = 0

            for feature, label in features_labels:
                outputs = model(feature)
                loss = criterion(outputs, label)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                _, predicted = outputs.max(1)
                total += label.size(0)
                correct += predicted.eq(label).sum().item()

            #train_loss /= len(train_loader)
            train_accuracy.append(correct / total)

    return sum(train_accuracy) / len(train_accuracy)


def feed_Data(chromosomes):
    input_size = 512  # Size of the ResNet output feature vector
    output_size = 10  # Number of classes in CIFAR-10 dataset

    # Preload feature extraction models
    feature_extraction_models = {
        1: models.resnet18(pretrained=True),
        2: models.resnet34(pretrained=True),
        3: models.vgg11(pretrained=True)
    }

    for i in range(len(chromosomes)):
        chromosome = chromosomes[i]
        if len(chromosome[hidden_layer]) == 0:
            model = MLP_0hidden(output_size, chromosome[type_activation_function])
        elif len(chromosome[hidden_layer]) == 1:
            model = MLP_1hidden(chromosome[hidden_layer], chromosome[type_activation_function])
        elif len(chromosome[hidden_layer]) == 2:
            model = MLP_2hidden(chromosome[hidden_layer], chromosome[type_activation_function])

        feature_extraction = feature_extraction_models[chromosome[type_feature_extraction]]
        feature_extraction = nn.Sequential(*list(feature_extraction.children())[:-1])  # Remove the last fully connected layer
        # Move feature extraction model to the same device as the model
        chromosomes[i][fitness] = train(model, feature_extraction, train_loader)

    return chromosomes

100%|██████████| 170498071/170498071 [00:01<00:00, 90545704.84it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
def evaluate_fitness_population(population):
    feed_Data(population)
    return population

In [ ]:
def select_parents(population):
    num_selected = int(crossover_rate*len(population))
    return population[:num_selected]

In [ ]:
def crossover_feature(parent1, parent2):
    parent1 , parent2 = parent2 , parent1
    return parent1 , parent2

In [ ]:
def crossover_layer_activation(parent1_layer, parent2_layer, parent1_active, parent2_active):
    parent1_len = len(parent1_layer)
    parent2_len = len(parent2_layer)
    
    if parent1_len - parent2_len == -1:
        value = parent2_layer.pop()
        parent1_layer.append(value)
        value = parent2_active.pop()
        parent1_active.append(value)

    elif parent1_len - parent2_len == 1:
        value = parent1_layer.pop()
        parent2_layer.append(value)
        value = parent1_active.pop()
        parent2_active.append(value)

    elif parent1_len == parent2_len and parent1_len != 0:
        parent1_layer[-1], parent2_layer[-1] = parent2_layer[-1], parent1_layer[-1]
        parent1_active[-1], parent2_active[-1] = parent2_active[-1], parent1_active[-1]

    elif parent1_len == 0 and parent2_len == 0:
        pass
    
    else:
        if parent2_len > parent1_len:
            value = parent2_layer.pop()
            parent1_layer.append(value)
            value = parent2_active.pop()
            parent1_active.append(value)
        else:
            value = parent1_layer.pop()
            parent2_layer.append(value)
            value = parent1_active.pop()
            parent2_active.append(value)
    
    
    return parent1_layer, parent2_layer, parent1_active, parent2_active

In [ ]:
def crossover(parents):
    offspring = list()
    for i in range(int(len(parents)/2)):
        child1 = dict()
        child2 = dict()

        child1[type_feature_extraction], child2[type_feature_extraction] = crossover_feature(parents[2*i][type_feature_extraction], parents[2*i+1][type_feature_extraction])
        child1[hidden_layer], child2[hidden_layer], child1[type_activation_function], child2[type_activation_function] = crossover_layer_activation(parents[2*i][hidden_layer], parents[2*i+1][hidden_layer],parents[2*i][type_activation_function], parents[2*i+1][type_activation_function])

        child1[fitness] = 0
        child2[fitness] = 0
        
        offspring.append(child1)
        offspring.append(child2)
    
    return offspring
        

In [ ]:
def mutation_feature(parent):
    parent = random.randint(1,3)
    return parent

In [ ]:
def mutation_activition(parent):
    index = random.randint(0, len(parent)-1)
    parent[index] = random.randint(1,2)
    return parent

In [ ]:
def mutation_layer(parent):
    index = random.randint(0, len(parent)-1)
    parent[index] = random.choice([10, 20, 30])
    return parent

In [ ]:
def mutate(offspring):
    for child in offspring:
        if random.uniform(0, 1) <= mutation_rate:
            child[type_feature_extraction] = mutation_feature(child[type_feature_extraction])
        if random.uniform(0, 1) <= mutation_rate:
            child[type_activation_function] = mutation_activition(child[type_activation_function])
        if random.uniform(0, 1) <= mutation_rate:
            child[hidden_layer] = mutation_layer(child[hidden_layer])
    
    return offspring

In [ ]:
def select_survivors(population, offspring):
    while(len(offspring) > 0):
        best_child = max(offspring, key=lambda x: x[fitness])
        bad_parent = min(population, key=lambda x: x[fitness])
        if best_child[fitness] > bad_parent[fitness]:
            population.remove(bad_parent)
            population.append(best_child)
            offspring.remove(best_child)
        else:
            break
    
    return population

In [ ]:
def genetic_algorithm():
    # Initialize population
    population = create_chromosomes(num_of_chromosomes)

    # Evaluate population fitness
    # fitness = accuracy of the model 
    evaluate_fitness_population(population)

    # Sort population
    population.sort(key = lambda x: x[fitness])

    print(population)

    # While termination condition is not met
    for _ in range(num_of_generations):
        # Select parents
        parents = select_parents(population)

        # Apply crossover / mutation operators
        offspring = crossover(parents)
        mutate(offspring)

        # Evaluate offspring
        evaluate_fitness_population(offspring)

        # Select survivors
        population = select_survivors(population, offspring)

        # Sort population
        population.sort(key = lambda x: x[fitness])

        print(population)
        print(max(population, key=lambda x: x[fitness]))
        mean = sum([x[fitness] for x in population])/len(population)
        chart_fitness.append(mean)

    chart()
    chart_fitness.clear()
    
    return max(population, key=lambda x: x[fitness])   # Return best individual in population

In [6]:
population = [{'feature extraction': 3, 'hidden layer': [30, 30], 'activation function': [2, 2, 2], 'fitness': 0.09709200000000001}, {'feature extraction': 3, 'hidden layer': [], 'activation function': [2], 'fitness': 0.09999040000000005}, {'feature extraction': 2, 'hidden layer': [20], 'activation function': [2, 2], 'fitness': 0.3952783999999999}, {'feature extraction': 1, 'hidden layer': [10], 'activation function': [1, 1], 'fitness': 0.42075919999999994}, {'feature extraction': 1, 'hidden layer': [30, 30], 'activation function': [1, 2, 1], 'fitness': 0.5034144}, {'feature extraction': 2, 'hidden layer': [30], 'activation function': [1, 2], 'fitness': 0.5189104}, {'feature extraction': 3, 'hidden layer': [30, 20], 'activation function': [2, 1, 2], 'fitness': 0.5197343999999999}, {'feature extraction': 2, 'hidden layer': [30], 'activation function': [1, 1], 'fitness': 0.5295344}, {'feature extraction': 3, 'hidden layer': [20, 30], 'activation function': [1, 1, 2], 'fitness': 0.6658688000000003}, {'feature extraction': 3, 'hidden layer': [20], 'activation function': [1, 1], 'fitness': 0.6825863999999998}]
print(max(population, key=lambda x: x['fitness']))
mean = sum([x['fitness'] for x in population])/len(population)
print(mean)

{'feature extraction': 3, 'hidden layer': [20], 'activation function': [1, 1], 'fitness': 0.6825863999999998}
0.44331687999999997


In [ ]:
x = genetic_algorithm()

print('Best NNA: ', x)